In [1]:
from fastembed import TextEmbedding

In [2]:
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [3]:
query = "I just discovered the course. Can I join now?"

In [4]:
q = list(model.embed(query))[0]

In [5]:
q.size

512

In [6]:
#Question 1
q.min()

np.float64(-0.11726373885183883)

In [7]:
import numpy as np
np.linalg.norm(q)

np.float64(1.0)

In [8]:
q.dot(q)

np.float64(1.0000000000000002)

In [9]:
q2 = list(model.embed("Can I still join the course after the start date?"))[0]

In [10]:
# Question 2
q2.dot(q)

np.float64(0.9008528895674548)

In [11]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [12]:
texts = [doc['text'] for doc in documents]

In [13]:
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

In [14]:
def find_best_match(text_list, query_text):

    text_embeddings = np.array(list(model.embed(text_list)))
    query_embedding = list(model.embed([query_text]))[0]

    # Compute cosine similarities
    cosine_similarities = text_embeddings.dot(query_embedding)

    # Get the index of the highest similarity
    best_index = np.argmax(cosine_similarities)

    return best_index

In [15]:
#Question 3
find_best_match(texts,query)

np.int64(1)

In [16]:
#Question 4
find_best_match(full_texts,query)

np.int64(0)

In [17]:
model = TextEmbedding(model_name="BAAI/bge-small-en-v1.5")
q5_1 = list(model.embed(query))
len(q5_1[0])

384

In [18]:
from fastembed import TextEmbedding
from fastembed.common.model_description import PoolingType, ModelSource

TextEmbedding.add_custom_model(
    model="intfloat/multilingual-e5-small-256",
    pooling=PoolingType.MEAN,
    normalization=True,
    sources=ModelSource(hf="intfloat/multilingual-e5-small"),  # can be used with an `url` to load files from a private storage
    dim=256,
    model_file="onnx/model.onnx",  # can be used to load an already supported model with another optimization or quantization, e.g. onnx/model_O4.onnx
)


In [19]:
model = TextEmbedding(model_name="intfloat/multilingual-e5-small-256")
q5_3 = list(model.embed(query))

In [20]:
q5_3[0].size

384

## Question 6

In [21]:
from qdrant_client import QdrantClient, models

In [22]:
qd_client = QdrantClient("http://localhost:6333")

In [23]:
EMBEDDING_DIMENSIONALITY = 384

In [24]:
model_handle = "BAAI/bge-small-en"

In [25]:
collection_name = "Homework-2"

In [26]:
if qd_client.collection_exists(collection_name=collection_name):
    qd_client.delete_collection(collection_name=collection_name)
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [27]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [28]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [29]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [30]:
len(points)

375

In [31]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [32]:
query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query,
        model=model_handle 
    ),
    limit=5,
    with_payload=True
)

In [34]:
#Question6
for point in query_points.points:
    print(point.score)

0.87031734
0.8691884
0.8683311
0.8576106
0.857154
